In [36]:
!pip install pmdarima
!pip install optuna
!pip install pandas-ta
!pip install catbost
!pip install prophet
!pip install finance-datareader

ERROR: Could not find a version that satisfies the requirement catbost (from versions: none)
ERROR: No matching distribution found for catbost


In [37]:
import pandas as pd
import numpy as np
import random
import os
from tqdm.auto import tqdm
from xgboost import XGBRegressor
import optuna
import pandas_ta as ta
from sklearn.metrics import mean_squared_error
from prophet import Prophet
from sklearn.model_selection import TimeSeriesSplit
from pmdarima.arima import auto_arima
import FinanceDataReader as fdr
import warnings
warnings.filterwarnings("ignore")

In [4]:
def seed_everything(seed):
  random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  np.random.seed(seed)

seed_everything(6789)

In [5]:
train = pd.read_csv("/content/drive/MyDrive/dacon_trading_bot/train.csv")

In [45]:
train["일자"] = pd.to_datetime(train["일자"], format="%Y%m%d")
train.tail()

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
987995,2023-05-30,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,2023-05-30,A000540,흥국화재,50218,3250,3255,3195,3215
987997,2023-05-30,A003280,흥아해운,130664,1344,1395,1340,1370
987998,2023-05-30,A037440,희림,141932,9170,9260,9170,9200
987999,2023-05-30,A238490,힘스,2611843,6410,8220,6300,8220


In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler, SequentialSampler
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTM_Trader(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
    super(LSTM_Trader, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.output_dim = output_dim
    self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)
    self.fc = nn.Linear(self.hidden_dim, self.output_dim)
  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out)
    return out

In [ ]:
unique_codes = train["종목코드"].unique()

results_df = pd.DataFrame(columns=["종목코드", "final_return"])

for idx, code in tqdm(enumerate(unique_codes[7:]), total=len(unique_codes)):
  print(f"===== processing data {idx+1}/2000 ======")
  start_date = "20000101"
  end_date = "20230530"
  stock = fdr.DataReader(code[1:], start=start_date, end=end_date)
  stock = stock.iloc[:, :-1]
  stock.rename(columns={"Open": "시가", "High":"고가", "Low": "저가", "Close": "종가", "Volume": "거래량"}, inplace=True)
  stock["일자"] = pd.to_datetime(stock.index, format="%Y%m%d")
  stock.drop(columns={"일자"}, inplace=True)
  close = stock["종가"].values
  X, Y = [], []
  for i in range(15, stock.shape[0]-15):
    cur_x = stock.iloc[i-15:i].values
    cur_y = (close[i+15] - close[i]) / close[i]
    X.append(cur_x)
    Y.append(cur_y)
  X = torch.tensor(np.array(X)).float()
  Y = torch.tensor(Y).float()
  X_test = stock.iloc[-15:].values
  X_test = torch.tensor(np.array(X_test)).float()
  X_test = torch.reshape(X_test, (-1, 15, 5))

  train_size = int(X.shape[0] * 0.9) # 9:1 split
  X_train = X[:train_size]
  Y_train = Y[:train_size]
  X_val = X[train_size:]
  Y_val = Y[train_size:]
  mu, std = torch.mean(X_train), torch.std(X_train)
  X_train = (X_train - mu) / std
  X_val = (X_val - mu) / std
  X_test = (X_test - mu) / std

  train_data = TensorDataset(X_train, Y_train)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

  val_data = TensorDataset(X_val, Y_val)
  val_sampler = SequentialSampler(val_data)
  val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=128)

  model = LSTM_Trader(input_dim=5, hidden_dim=50, num_layers=2, output_dim=1).to(device)
  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
  epochs = 40
  total_steps = len(train_dataloader) * epochs
  loss_func = nn.SmoothL1Loss()
  best_val_loss = 99999999999
  best_model = None
  model.zero_grad()

  val_losses = []
  for epoch in tqdm(range(epochs), total=epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
      batch = tuple(t.to(device) for t in batch)
      b_seqs, b_targets = batch
      outputs = model(b_seqs)
      loss = loss_func(outputs, b_targets)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      model.zero_grad()
    val_loss = 0
    model.eval()
    for step, batch in enumerate(val_dataloader):
      batch = tuple(t.to(device) for t in batch)
      b_seqs, b_targets = batch
      with torch.no_grad():
        outputs = model(b_seqs)
      loss = loss_func(outputs, b_targets)
      val_loss += loss.item()
    avg_val_loss = val_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
    if avg_val_loss < best_val_loss:
      best_val_loss = avg_val_loss
      best_model = model

  best_model.eval()
  with torch.no_grad():
    final_return = best_model(X_test.to(device)).detach().cpu().numpy()[0][0]
  results_df = results_df.append({"종목코드":code, "final_return":final_return}, ignore_index=True)
  if idx == 10:
    break

  0%|          | 0/2000 [00:00<?, ?it/s]

===== processing data 1/2000 ======


  0%|          | 0/40 [00:00<?, ?it/s]

===== processing data 2/2000 ======


  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

===== processing data 4/2000 ======


  0%|          | 0/40 [00:00<?, ?it/s]

===== processing data 5/2000 ======


  0%|          | 0/40 [00:00<?, ?it/s]

In [92]:
results_df["순위"] = results_df["final_return"].rank(method="first").astype("int")
sample_submission = pd.read_csv("/content/drive/MyDrive/dacon_trading_bot/sample_submission.csv")
lstm_submission = sample_submission[["종목코드"]].merge(results_df[["종목코드", "순위"]], on="종목코드", how="left")
lstm_submission.to_csv("/content/drive/MyDrive/dacon_trading_bot/lstm_baseline.csv", index=False)

In [93]:
results_df

,종목코드,final_return,순위
0,A282330,[0.040183797],11
1,A126600,[-0.00013687462],1
2,A138930,[0.006523736],3
3,A001460,[0.010348321],6
4,A013720,[0.022268265],9
5,A001040,[0.012817081],7
6,A079160,[0.0024542846],2
7,A035760,[0.009267246],5
8,A311690,[0.034805063],10
9,A000120,[0.008775599],4


In [ ]:
plt.plot(val_losses)

In [87]:
results_df

,종목코드,final_return,순위
0,A282330,[0.013759688],10
1,A126600,[-0.000577122],2
2,A138930,[-0.0035121394],1
3,A001460,[0.008928761],7
4,A013720,[0.018268337],11
5,A001040,[0.0076376647],5
6,A079160,[0.005124956],4
7,A035760,[0.008601759],6
8,A311690,[0.0029703535],3
9,A000120,[0.011639856],8
